In [1]:
%cd ..

C:\Python\weldx


In [2]:
import numpy as np

import asdf
import pandas as pd
import pint
import sympy
import weldx
import weldx.measurement as msm
import weldx.transformations as tf
import xarray as xr
from matplotlib import pyplot as plt
from weldx import Q_
from weldx.asdf.extension import WeldxAsdfExtension, WeldxExtension

# Handling measurements
## Overview
In this short example we use welding voltage and current measurements to show how to describe and store measurements with associated measurement chains. This includes describing the measurement equipment and its metadata, describing all the relevant transformation steps from raw-data to the final output and the data itself. The final result is a `MeasurementChain` that should be easy to follow and represent the complete data processing pipeline.

## Generating the measurement data
We start by creating some "dummy" datasets that represent the current and voltage measurements.
In a real application, these would be the datasets that we would copy from our measurement equipment (e.g. downloaded form a HKS-WeldQAS, oscilloscope or similar systems).
The values in these dataset represent the actual physical current and voltage data in A and V.

In [3]:
# a small helper function to generate a sine signal with its time axis. This supports using pint.Quantities !
def sine(f, amp, t_end, samples=None):
    """Generate a sine with frequency f, amplitude amp and timespan t_end"""
    if samples is None:  # default to 50 samples per period
        samples = int((t_end * 2 * Q_("Hz") * 50).to_reduced_units()) + 1
    om = 2 * np.pi * f
    t = np.linspace(0 * Q_("s"), t_end, num=samples)
    y = amp * np.sin(t * om)
    return y, t

In [4]:
I, time = sine(f=1.5 * Q_("Hz"), amp=50 * Q_("A"), t_end=10 * Q_("s"))
time = pd.TimedeltaIndex(data=time.magnitude, unit="s") + pd.Timestamp("2020-06-01")

current_data = msm.Data(
    name="Welding current", data=xr.DataArray(I, dims=["time"], coords={"time": time})
)
current_data

Data(name='Welding current', data=<xarray.DataArray (time: 1001)>
<Quantity([ 0.00000000e+00  4.70541567e+00  9.36906573e+00 ... -9.36906573e+00
 -4.70541567e+00 -5.38968388e-13], 'ampere')>
Coordinates:
  * time     (time) datetime64[ns] 2020-06-01 ... 2020-06-01T00:00:10)

In [5]:
V, time = sine(f=1.5 * Q_("Hz"), amp=20 * Q_("V"), t_end=10 * Q_("s"))
time = pd.TimedeltaIndex(data=time.magnitude, unit="s") + pd.Timestamp("2020-06-01")

voltage_data = msm.Data(
    name="Welding voltage", data=xr.DataArray(V, dims=["time"], coords={"time": time})
)
voltage_data

Data(name='Welding voltage', data=<xarray.DataArray (time: 1001)>
<Quantity([ 0.00000000e+00  1.88216627e+00  3.74762629e+00 ... -3.74762629e+00
 -1.88216627e+00 -2.15587355e-13], 'volt')>
Coordinates:
  * time     (time) datetime64[ns] 2020-06-01 ... 2020-06-01T00:00:10)

It is important to note the type and structure of the `current_data` and `voltage_data` datasets:
- they are created as `xarray.DataArrays`
- the data itself is a `pint.Quantity` i.e. a numpy array with associated unit. For the current measurement this is `ampere`, the voltage is given in `volt`. Using quantities is an important core concept of measurements !
- each `DataArray` has a `time` dimension and coordinate using numpy datetime formats.

## Equipment and Software
Next, let's define some of the equipment and software that is used throughout the measurement chain. We will use and add more information to these objects later.

In out example, two types of hardware equipment are used:
- The [`HKS P1000-S3`](https://hks-prozesstechnik.de/en/sensors-2/) is a standard welding process sensor that detects the welding voltage and current using a hall sensor. The result is output as two analog signals scaled to +/- 10 V.
- The [`Beckhoff ELM3002-0000`](https://www.beckhoff.com/ELM3002/) is a fieldbus AD-converter terminal that picks up the analog signals of the HKS Sensor and transmits them digitally to the control software.

The final piece involved in the measurement chain is the Software used to record, scale and save both the welding current and voltage measurements. We define the software version and name used during the example using built-in ASDF types.

In [6]:
HKS_sensor = msm.GenericEquipment(name="HKS P1000-S3")

BH_ELM = msm.GenericEquipment(name="Beckhoff ELM3002-0000")

In [7]:
from asdf.tags.core import Software

twincat_scope = Software(name="Beckhoff TwinCAT ScopeView", version="3.4.3143")

## Defining a measurement chain: current measurement
Now we define the missing elements of our measurement chain and bundle everything together. A core concept of the chain are `signals` that go in and out of `transformations` which define mathematical operations of the signals, forming the chain.

Each measurement chain starts with a `Source` signal. This is the point where our physical process or quantity is initially detected by a sensor. We define a `measurement.Source` object by giving a `name`, defining the `Signal` type and an `error` representing the uncertainty attached to the signal source. In a way, a `Source` is a special type of `transformation` in that it only has an `output_signal` but no `input_signal`.

For our current measurement, the source outputs an **analog Signal** of unit **V**. Since there is no recording of this measurement we do not provide the data. According to the spec sheet of the sensor the measurement error in this initial step is **0.1 %** which can be documented using the `Error` property (again using quantities).

In [8]:
src_current = msm.Source(
    name="Current Sensor",
    output_signal=msm.Signal(signal_type="analog", unit="V", data=None),
    error=msm.Error(Q_(0.1, "percent")),
)

We associate the current measurement source with the HKS sensor by adding it to its list of sources. We now have the start of our measurement chain defined.

In [9]:
HKS_sensor.sources = []
HKS_sensor.sources.append(src_current)

The next step in the chain is picking up the analog voltage signal from our source with the Beckhoff AD converter terminal which transform the signal into an internal signed integer value. The formula describing this linear transformation with input `x` is
```
a * x + b
32768 / (10 V) * x  + 0
```
We express this signal transformation as an analytical formula created with the `sympy` package. Based on the above formula we also define the static parameters `a` and `b` in the `MathematicalExpression`. Note that we use quantities here as well !
Since our result is a dimensionless integer `a` has the unit **1/V** and `b` is dimensionless which we indicate with `""`.

In [10]:
from weldx.asdf.tags.weldx.core.mathematical_expression import MathematicalExpression

[a, x, b] = sympy.symbols("a x b")
current_AD_func = MathematicalExpression(a * x + b)
current_AD_func.set_parameter("a", Q_(32768.0 / 10.0, "1/V"))
current_AD_func.set_parameter("b", Q_(0.0, ""))

Now that we have the transform function we can define our `DataTransformation`. The `input_signal` of our transformation is the `src_current.output_signal` object from our source signal defined earlier.
Our new Transformation outputs a new **dimensionless** signal of type **digital**. Once again, since we have no data record of this we do not assign any data object to the signal. The Beckhoff AD converter lists the measurement Error at **0.01 %**.

In [11]:
current_AD_transform = msm.DataTransformation(
    name="AD conversion current measurement",
    input_signal=src_current.output_signal,
    output_signal=msm.Signal("digital", "", data=None),
    error=msm.Error(Q_(0.01, "percent")),
    func=current_AD_func,
)

We also associate the transformation to the Beckhoff equipment:

In [12]:
BH_ELM.data_transformations = []
BH_ELM.data_transformations.append(current_AD_transform)

Similar to the AD conversion, we add the final step of our signal processing chain: digitally converting the signal to the final physical representation of the welding current. The current calibration formula from our integer values to the real current values is as follows:
```
1000 A / 32768 * x  + 0 A
```
Put into a new `sympy` expression:

In [13]:
# define current output calibration expression and transformation
current_calib_func = MathematicalExpression(a * x + b)
current_calib_func.set_parameter("a", Q_(1000.0 / 32768.0, "A"))
current_calib_func.set_parameter("b", Q_(0.0, "A"))

We create the final transformation step:
- the input signal now is the output signal of the previous AD conversion
- the output signal is our final current measurement representation
- we add the our measurement data to this signal !
- we add the software as a meta field to the signal transformation

In [14]:
current_calib_transform = msm.DataTransformation(
    name="Calibration current measurement",
    input_signal=current_AD_transform.output_signal,
    output_signal=msm.Signal("digital", "A", data=current_data),
    error=msm.Error(0.0),
    func=current_calib_func,
    meta=twincat_scope,
)

Now that all transformation steps are defined, we can create our `MeasurementChain` object. We pass on the initial source as well as all transformation steps in order.

In [15]:
welding_current_chain = msm.MeasurementChain(
    name="welding current measurement chain",
    data_source=src_current,
    data_processors=[current_AD_transform, current_calib_transform],
)

Finally the `Measurement` is our measurement chain with another link to the data.

In [16]:
welding_current = msm.Measurement(
    name="welding current measurement",
    data=[current_data],
    measurement_chain=welding_current_chain,
)

## voltage measurement
We follow the same procedure described in the current measurement here :-)

In [17]:
src_voltage = msm.Source(
    name="Voltage Sensor",
    output_signal=msm.Signal("analog", "V", data=None),
    error=msm.Error(Q_(0.1, "percent")),
)

HKS_sensor.sources.append(src_voltage)

# define AD conversion expression and transformation step
[a, x, b] = sympy.symbols("a x b")
voltage_ad_func = MathematicalExpression(a * x + b)
voltage_ad_func.set_parameter("a", Q_(32768.0 / 10.0, "1/V"))
voltage_ad_func.set_parameter("b", Q_(0.0, ""))

voltage_AD_transform = msm.DataTransformation(
    name="AD conversion voltage measurement",
    input_signal=src_voltage.output_signal,
    output_signal=msm.Signal("digital", "", data=None),
    error=msm.Error(Q_(0.01, "percent")),
    func=voltage_ad_func,
)

HKS_sensor.data_transformations.append(voltage_AD_transform)

# define voltage output calibration expression and transformation
voltage_calib_func = MathematicalExpression(a * x + b)
voltage_calib_func.set_parameter("a", Q_(100.0 / 32768.0, "V"))
voltage_calib_func.set_parameter("b", Q_(0.0, "V"))

voltage_calib_transform = msm.DataTransformation(
    name="Calibration voltage measurement",
    input_signal=voltage_AD_transform.output_signal,
    output_signal=msm.Signal("digital", "V", data=voltage_data),
    error=msm.Error(0.0),
    func=voltage_calib_func,
    meta=twincat_scope,
)


welding_voltage_chain = msm.MeasurementChain(
    name="welding voltage measurement chain",
    data_source=src_voltage,
    data_processors=[voltage_AD_transform, voltage_calib_transform],
)

welding_voltage = msm.Measurement(
    name="welding voltage measurement",
    data=[voltage_data],
    measurement_chain=welding_voltage_chain,
)

## Coordinate Systems

Most data does not make much sense without being able to determine where it was recorded in relation to a specimen or other measurement spots. Therefore, we define coordinate systems and their orientations towards each other. The basic principles are already explained in the transformation tutorials, so we will just define some coordinate systems without further explanation. To keep things simple, no time dependent coordinates are considered. 

In [18]:
lcs_specimen_in_root = tf.LocalCoordinateSystem(
    coordinates=Q_(np.asarray([100, 75, 0]), "mm")
)
lcs_flange_in_root = tf.LocalCoordinateSystem(
    orientation=tf.rotation_matrix_x(np.pi / 2),
    coordinates=Q_(np.asarray([115, -10, 140]), "mm"),
)
lcs_torch_in_flange = tf.LocalCoordinateSystem(
    coordinates=Q_(np.asarray([100, 75, 0]), "mm")
)

coordinate_systems = tf.CoordinateSystemManager("root")
coordinate_systems.add_coordinate_system("specimen", "root", lcs_specimen_in_root)
coordinate_systems.add_coordinate_system("flange", "root", lcs_flange_in_root)
coordinate_systems.add_coordinate_system("torch", "flange", lcs_torch_in_flange)

> **TODO:** Connect data to coordinate systems

## Writing to ASDF
Once we have define all object we can write them to a ASDF file. To make the file easier to read we place some elements earlier in the tree.

In [19]:
equipment = [HKS_sensor, BH_ELM]
measurement_data = [current_data, voltage_data]
measurements = [welding_current, welding_voltage]

tree = {
    "coordinate_systems": coordinate_systems,
    "equipment": equipment,
    "data": measurement_data,
    "measurements": measurements,
    # "expression": expr_01,
    # "measurement_chains": measurement_chains,
    # "data_sources": sources,
    # "data_processors": processors,
}
with asdf.AsdfFile(tree, extensions=[WeldxExtension(), WeldxAsdfExtension()]) as f:
    f.write_to("tutorials/measurement_example.asdf")